In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
import random

# Load datasets
jobs_df = pd.read_csv("/content/drive/MyDrive/dataset/job_skills.csv")
courses_df = pd.read_csv("/content/drive/MyDrive/dataset/coursea_data.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
jobs_df.head()

,Company,Title,Category,Location,Responsibilities,Minimum Qualifications,Preferred Qualifications
0,Google,Google Cloud Program Manager,Program Management,Singapore,"Shape, shepherd, ship, and show technical prog...",BA/BS degree or equivalent practical experienc...,Experience in the business technology market a...
1,Google,"Supplier Development Engineer (SDE), Cable/Con...",Manufacturing & Supply Chain,"Shanghai, China",Drive cross-functional activities in the suppl...,BS degree in an Engineering discipline or equi...,"BSEE, BSME or BSIE degree.\nExperience of usin..."
2,Google,"Data Analyst, Product and Tools Operations, Go...",Technical Solutions,"New York, NY, United States",Collect and analyze data to draw insight and i...,"Bachelor’s degree in Business, Economics, Stat...",Experience partnering or consulting cross-func...
3,Google,"Developer Advocate, Partner Engineering",Developer Relations,"Mountain View, CA, United States","Work one-on-one with the top Android, iOS, and...",BA/BS degree in Computer Science or equivalent...,"Experience as a software developer, architect,..."
4,Google,"Program Manager, Audio Visual (AV) Deployments",Program Management,"Sunnyvale, CA, United States",Plan requirements with internal customers.\nPr...,BA/BS degree or equivalent practical experienc...,CTS Certification.\nExperience in the construc...


In [ ]:
courses_df.head()

,Unnamed: 0,course_title,course_organization,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled
0,134,(ISC)² Systems Security Certified Practitioner...,(ISC)²,SPECIALIZATION,4.7,Beginner,5.3k
1,743,A Crash Course in Causality: Inferring Causal...,University of Pennsylvania,COURSE,4.7,Intermediate,17k
2,874,A Crash Course in Data Science,Johns Hopkins University,COURSE,4.5,Mixed,130k
3,413,A Law Student's Toolkit,Yale University,COURSE,4.7,Mixed,91k
4,635,A Life of Happiness and Fulfillment,Indian School of Business,COURSE,4.8,Mixed,320k


In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
import random
import os

# --- Configuration ---
DATASET_PATH = "/content/drive/MyDrive/dataset/" # Adjust this path to your dataset location
OUTPUT_DIR = "." # Current directory. Change if you want to save datasets elsewhere.

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Load your datasets ---
try:
    # UPDATED: Loading new datasets
    jobs_df = pd.read_csv(os.path.join(DATASET_PATH, "job_skills.csv"))
    courses_df = pd.read_csv(os.path.join(DATASET_PATH, "coursea_data.csv"))
    print("New datasets (job_skills.csv, coursea_data.csv) loaded successfully.")
except FileNotFoundError as e:
    print(f"Error: One or more dataset files not found. Please check the DATASET_PATH and filenames. {e}")
    exit()
except Exception as e:
    print(f"Error loading datasets: {e}")
    exit()

qa_data = []
current_id = 0

# --- Helper to create a QA entry ---
def add_qa_entry(qa_list, title, context, question, answer_text):
    global current_id
    if not context or not question or not answer_text:
        return # Skip if any essential part is missing

    # Ensure the answer_text is actually IN the context
    ans_start = context.find(answer_text)
    if ans_start == -1:
        # If not found, print a warning and skip to avoid bad training data
        # print(f"Warning: Answer '{answer_text}' not found in context for Q: '{question}'. Skipping.")
        return

    qa_list.append({
        "id": str(current_id),
        "title": title,
        "context": context,
        "question": question,
        "answers": {"text": [answer_text], "answer_start": [ans_start]}
    })
    current_id += 1

# --- Process Jobs Data ---
for idx, row in jobs_df.iterrows():
    company = str(row.get('Company')).strip() if pd.notna(row.get('Company')) else ""
    title = str(row.get('Title')).strip() if pd.notna(row.get('Title')) else ""
    category = str(row.get('Category')).strip() if pd.notna(row.get('Category')) else ""
    location = str(row.get('Location')).strip() if pd.notna(row.get('Location')) else ""
    responsibilities = str(row.get('Responsibilities')).strip() if pd.notna(row.get('Responsibilities')) else ""
    min_qual = str(row.get('Minimum Qualifications')).strip() if pd.notna(row.get('Minimum Qualifications')) else ""
    pref_qual = str(row.get('Preferred Qualifications')).strip() if pd.notna(row.get('Preferred Qualifications')) else ""

    if not title and not responsibilities: # Core info needed
        continue

    # Limit context size for fine-tuning to avoid truncation issues during training
    # A character limit is a rough proxy, tokenizers are more precise.
    max_desc_len = 400
    truncated_responsibilities = responsibilities[:max_desc_len] + "..." if len(responsibilities) > max_desc_len else responsibilities
    truncated_min_qual = min_qual[:max_desc_len] + "..." if len(min_qual) > max_desc_len else min_qual
    truncated_pref_qual = pref_qual[:max_desc_len] + "..." if len(pref_qual) > max_desc_len else pref_qual


    context_parts = []
    if title: context_parts.append(f"Job Title: {title}")
    if company: context_parts.append(f"Company: {company}")
    if category: context_parts.append(f"Category: {category}")
    if location: context_parts.append(f"Location: {location}")
    if truncated_responsibilities: context_parts.append(f"Responsibilities: {truncated_responsibilities}")
    if truncated_min_qual: context_parts.append(f"Minimum Qualifications: {truncated_min_qual}")
    if truncated_pref_qual: context_parts.append(f"Preferred Qualifications: {truncated_pref_qual}")

    context = ". ".join(filter(None, context_parts))

    if not context.strip():
        continue

    # Q1: What is this job about? -> A: Responsibilities
    if title and truncated_responsibilities:
        question1 = f"What are the responsibilities of a '{title}' at '{company}'?" if company else f"What are the responsibilities of a '{title}'?"
        add_qa_entry(qa_data, "Job Responsibilities", context, question1, truncated_responsibilities)

    # Q2: What qualifications are needed? -> A: Minimum Qualifications
    if title and truncated_min_qual:
        question2 = f"What are the minimum qualifications for the '{title}' role?"
        add_qa_entry(qa_data, "Job Minimum Qualifications", context, question2, truncated_min_qual)

    # Q3: What preferred qualifications are there? -> A: Preferred Qualifications
    if title and truncated_pref_qual:
        question3 = f"What are the preferred qualifications for the '{title}' role?"
        add_qa_entry(qa_data, "Job Preferred Qualifications", context, question3, truncated_pref_qual)

    # Q4: Tell me about this job. -> A: Job Title + a bit of description
    if title and context:
        summary_answer = f"{title} at {company}. Responsibilities include {truncated_responsibilities[:100]}..." if company else f"{title}. Responsibilities include {truncated_responsibilities[:100]}..."
        # Ensure summary_answer is a subset of context for training
        if context.find(summary_answer) == -1: # Adjust if needed for exact span matching
            summary_answer = title # Fallback to just title

        if context.find(summary_answer) != -1:
             add_qa_entry(qa_data, "Job Overview", context, f"Tell me about the '{title}' job.", summary_answer)


# --- Process Courses Data ---
for idx, row in courses_df.iterrows():
    course_title = str(row.get('course_title')).strip() if pd.notna(row.get('course_title')) else ""
    course_organization = str(row.get('course_organization')).strip() if pd.notna(row.get('course_organization')) else ""
    course_certificate_type = str(row.get('course_Certificate_type')).strip() if pd.notna(row.get('course_Certificate_type')) else ""
    course_rating = str(row.get('course_rating')).strip() if pd.notna(row.get('course_rating')) else ""
    course_difficulty = str(row.get('course_difficulty')).strip() if pd.notna(row.get('course_difficulty')) else ""
    course_students_enrolled = str(row.get('course_students_enrolled')).strip() if pd.notna(row.get('course_students_enrolled')) else ""

    if not course_title:
        continue

    context_parts = []
    if course_title: context_parts.append(f"Course Title: {course_title}")
    if course_organization: context_parts.append(f"Organization: {course_organization}")
    if course_certificate_type: context_parts.append(f"Certificate Type: {course_certificate_type}")
    if course_rating: context_parts.append(f"Rating: {course_rating}")
    if course_difficulty: context_parts.append(f"Difficulty: {course_difficulty}")
    if course_students_enrolled: context_parts.append(f"Students Enrolled: {course_students_enrolled}")
    # Add other relevant columns if they exist and are important for QA
    # For example: if you had 'course_description' or 'skills_covered' in coursea_data.csv, add them here.
    # For now, based on your head(), we use the provided columns.

    context = ". ".join(filter(None, context_parts))

    if not context.strip():
        continue

    # Q1: What is this course about? -> A: Course Title (as primary identifier) or more details
    if course_title:
        question1 = f"Tell me about the course '{course_title}'."
        add_qa_entry(qa_data, "Course Overview", context, question1, course_title) # Or a combo of details

    # Q2: Who offers this course? -> A: Organization
    if course_title and course_organization:
        question2 = f"Who offers the '{course_title}' course?"
        add_qa_entry(qa_data, "Course Organization", context, question2, course_organization)

    # Q3: What is the difficulty/rating/students of this course?
    if course_title and course_difficulty:
        question3 = f"What is the difficulty of '{course_title}'?"
        add_qa_entry(qa_data, "Course Difficulty", context, question3, course_difficulty)

    if course_title and course_rating:
        question4 = f"What is the rating of '{course_title}'?"
        add_qa_entry(qa_data, "Course Rating", context, question4, course_rating)

    # Q4: Find courses on X (using course title/organization for answers)
    if course_title:
        questions_by_topic = [
            f"Find courses related to {course_title.lower().split(':')[0].strip()}.", # Generic from title
            f"Recommend a course on {course_title.lower().split(':')[0].strip()} offered by {course_organization}.",
        ]
        # Assuming the Course Title is the most relevant answer for these general topic queries
        add_qa_entry(qa_data, "Course by Topic", context, random.choice(questions_by_topic), course_title)


# Create Hugging Face Dataset
if not qa_data:
    print("No QA data generated. Please check your input CSVs and data processing logic.")
    exit()

hf_dataset = Dataset.from_list(qa_data)

# Split into train and test sets
if len(hf_dataset) > 10: # Ensure enough data for a meaningful split
    train_test_split = hf_dataset.train_test_split(test_size=0.1, seed=42)
    train_dataset = train_test_split['train']
    test_dataset = train_test_split['test']
else:
    print(f"Warning: Not enough data ({len(hf_dataset)} samples) to create a test set. Using all data for training.")
    train_dataset = hf_dataset
    test_dataset = hf_dataset.select(range(min(1, len(hf_dataset)))) # Empty or single sample test set

print(f"Total QA pairs generated: {len(qa_data)}")
print(f"Training set size: {len(train_dataset)}")
print(f"Test set size: {len(test_dataset)}")

train_dataset.save_to_disk(os.path.join(OUTPUT_DIR, "qa_train_dataset_job_course")) # Specific name
test_dataset.save_to_disk(os.path.join(OUTPUT_DIR, "qa_test_dataset_job_course")) # Specific name

print("\nSample from training dataset:")
print(train_dataset[0])
print("\nQA data preparation complete. You can now proceed to fine-tuning.")

New datasets (job_skills.csv, coursea_data.csv) loaded successfully.
Total QA pairs generated: 9412
Training set size: 8470
Test set size: 942


Saving the dataset (0/1 shards):   0%|          | 0/8470 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/942 [00:00<?, ? examples/s]


Sample from training dataset:
{'id': '7646', 'title': 'Course by Topic', 'context': 'Course Title: Julia Scientific Programming. Organization: University of Cape Town. Certificate Type: COURSE. Rating: 4.5. Difficulty: Beginner. Students Enrolled: 22k', 'question': 'Find courses related to julia scientific programming.', 'answers': {'answer_start': [14], 'text': ['Julia Scientific Programming']}}

QA data preparation complete. You can now proceed to fine-tuning.


In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_from_disk
import torch

# --- Configuration ---
# Ensure these paths match where you saved them in prepare_qa_data_job_course.py
TRAIN_DATASET_PATH = "qa_train_dataset_job_course"
TEST_DATASET_PATH = "qa_test_dataset_job_course"
MODEL_OUTPUT_DIR = "./qa_model_output_job_course" # Directory to save checkpoints

# Load the datasets
train_dataset = load_from_disk(TRAIN_DATASET_PATH)
test_dataset = load_from_disk(TEST_DATASET_PATH)

# Choose your model checkpoint (Extractive QA model)
# 'distilbert-base-uncased' is a good starting point (fast, decent performance)
# Consider 'roberta-base' or 'microsoft/deberta-v3-base' for potentially better accuracy (larger models)
model_checkpoint = "distilbert-base-uncased"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

# --- Preprocessing Function for QA Task ---
# This function tokenizes the input and finds the start/end positions of the answer span.
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = examples["context"]

    # Tokenize questions and contexts, handling truncation for long contexts
    # return_offsets_mapping is crucial for mapping tokens back to original text for answer spans
    inputs = tokenizer(
        questions,
        contexts,
        max_length=512, # Max sequence length for DistilBERT. Adjust for other models.
        truncation="only_second", # Truncate context if too long
        return_offsets_mapping=True, # Important for mapping token indices to char indices
        padding="max_length",
        return_overflowing_tokens=True # Add this to get overflow_to_sample_mapping
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])

        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context in the tokenized sequence
        idx = 0
        while sequence_ids[idx] != 1: # 0 for question, 1 for context, None for special tokens
            idx += 1
        context_start_token_idx = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end_token_idx = idx - 1

        # If the answer is not fully in the context (due to truncation), label it (0, 0)
        # This teaches the model to predict 'no answer' if it can't find it
        if offset[context_start_token_idx][0] > end_char or offset[context_end_token_idx][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise, find the token positions of the answer
            idx = context_start_token_idx
            while idx <= context_end_token_idx and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end_token_idx
            while idx >= context_start_token_idx and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Apply preprocessing to datasets (use num_proc for faster processing if you have multiple CPU cores)
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names, num_proc=os.cpu_count() or 1)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=test_dataset.column_names, num_proc=os.cpu_count() or 1)


# Define training arguments
# Adjust these based on your hardware (GPU availability), dataset size, and desired training time.
training_args = TrainingArguments(
    output_dir=MODEL_OUTPUT_DIR, # Directory to save checkpoints and logs
    eval_strategy="epoch", # Evaluate at the end of each epoch
    save_strategy="epoch", # Save checkpoint at the end of each epoch to match eval_strategy
    learning_rate=2e-5, # Common learning rate for fine-tuning
    per_device_train_batch_size=8, # Adjust based on GPU memory. Increase if GPU has more memory.
    per_device_eval_batch_size=8,  # Adjust based on GPU memory.
    num_train_epochs=3, # Number of passes over the training data. Increase if loss still decreasing.
    weight_decay=0.01, # Regularization to prevent overfitting
    fp16=torch.cuda.is_available(), # Enable mixed precision training if GPU is available
    save_total_limit=1, # Only save the best model checkpoint
    load_best_model_at_end=True, # Load the best model found during training
    metric_for_best_model="eval_loss", # Metric to determine the best model
    push_to_hub=False, # Set to True if you want to upload to Hugging Face Hub (requires login)
    report_to="none", # Disable logging to external services (wandb, mlflow, etc.)
)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map (num_proc=2):   0%|          | 0/8470 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map (num_proc=2):   0%|          | 0/942 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer, # Pass tokenizer to Trainer for proper padding/collating
)

# Fine-tune the model
print("Starting fine-tuning...")
trainer.train()
print("Fine-tuning complete.")

/tmp/ipython-input-11-1869811995.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting fine-tuning...


Epoch,Training Loss,Validation Loss
1,0.032000,0.009271
2,0.010800,0.003890
3,0.003900,0.003225


Fine-tuning complete.


In [ ]:
# Save the fine-tuned model and tokenizer
FINE_TUNED_MODEL_PATH = "/content/drive/MyDrive/fine_tuned_qa_model_job_course" # Final saved model path
model.save_pretrained(FINE_TUNED_MODEL_PATH)
tokenizer.save_pretrained(FINE_TUNED_MODEL_PATH)
print(f"Fine-tuned model saved to {FINE_TUNED_MODEL_PATH}")

Fine-tuned model saved to /content/drive/MyDrive/fine_tuned_qa_model_job_course
